## Домашнее задание по свёрточным сетям

Сутью домашнего задания является последовательная реализация базовых операций, применяемых в свёрточных сетях с использованием операций над тензорами PyTorch, но без применения модулей torch.nn. Студенты должны самостоятельно реализовать как прямой и обратный проходы слоёв, так и классы нейронных сетей.

Правильность выполнения задания будет проверяться идентичностью прохождения процесса обучения в тех же архитектурах, выполненных с применением модулей и алгоритмов PyTorch.

### Задание 1

В первом задании требуется реализовать классы двухмерной свёртки **Conv**, линейного слоя **Fc**, алгоритм обучения нейронной сети **SGD**, функцию активации **ReLU**, **Softmax** и функцию эмпирического риска **CrossEntropyLoss**.

Свёрточные и полносвязные слои должны реализовывать операцию сдвига (*bias*, *b*). 
Сверить формулы прямого прохода можно в документации по [Conv2d](https://pytorch.org/docs/stable/generated/torch.nn.Conv2d.html) и [Linear](https://pytorch.org/docs/stable/generated/torch.nn.Linear.html).

Далее, следует реализовать класс модели, который включает в себя реализованные выше компоненты. Требуется повторить параметры обучения и архитектуру, реализованную ниже с помощью torch.nn модулей. Критерием правильности решения будет совпадение значений эмпирического риска при обучении обеих реализаций сетей на одних и тех же данных, с теми же параметрами и с одинаковыми начальными инициализациями весов.

Данные для обучения состоят из 4 примеров вертикальных и горизонтальных линий. 


In [ ]:
import torch
from torch import nn
import numpy as np
import matplotlib.pyplot as plt
import torch.optim
import torch.nn.functional as F
from tqdm import trange
import pandas as pd
import math
import copy

_ = torch.manual_seed(1)

In [ ]:
def one_hot_encoding(y, num_classes):
    N = y.shape[0]
    Z = torch.zeros((N, num_classes))
    Z[torch.arange(N), y] = 1
    return Z

In [ ]:
# Обучение будет вестись на следующих данных
def get_data():
    # Задача классификации вертикальных и горизонтальных линий
    # Данные
    vert1 = [[0, 0, 250, 0, 0],
             [0, 0, 250, 0, 0],
             [0, 0, 250, 0, 0],
             [0, 0, 250, 0, 0],
             [0, 0, 250, 0, 0]]

    vert2 = [[0, 0, 220, 40, 0],
             [0, 0, 250, 10, 0],
             [0, 0, 250, 0, 0],
             [0, 10, 250, 0, 0],
             [0, 40, 220, 0, 0]]

    hor1 = [[0, 0, 0, 0, 0],
            [0, 0, 0, 0, 0],
            [250, 250, 250, 250, 250],
            [0, 0, 0, 0, 0],
            [0, 0, 0, 0, 0]]

    hor2 = [[0, 0, 0, 0, 0],
            [0, 0, 0, 0, 20],
            [220, 250, 250, 250, 200],
            [10, 0, 0, 0, 0],
            [0, 0, 0, 0, 0]]

    data = [vert1, vert2, hor1, hor2]
    labels = [0, 0, 1, 1]
    
    train_x = torch.tensor(data, dtype=torch.float32).unsqueeze(1)
    labels = torch.tensor(labels, dtype=torch.long)

    return train_x, labels

train_x, labels = get_data()
print(f'Train data shape [Batch, Channels, Height, Width] = {train_x.shape}')
print(f'Labels shape = {labels.shape}')


train_mean = torch.mean(train_x)
train_std = torch.std(train_x)
batch = (train_x - train_mean) / train_std

print(f'Mean and standard deviation before normalization = {train_mean.item():.2f}, {train_std.item():.2f}')
print(f'Mean and standard deviation after normalization = {torch.mean(batch).item():.2f}, {torch.std(batch).item():.2f}')

print('Train batch')
for img in batch:
    plt.imshow(img.squeeze(0), cmap='Greys')  # Цвета инвертированы. Чем темнее, тем значение пикселя больше
    plt.show() 

Реализуем эталонную модель **TorchGradientModel**, состоящую из следующих модулей:
- Сверточный слой с 4 фильтрами размера $5\times5$;
- Функция активации [ReLU](https://pytorch.org/docs/stable/generated/torch.nn.ReLU.html)
- Линейный слой с 2 выходными нейронами для классов горизонтальной и вертикальной линий

Вопрос:
Каким ещё образом можно осуществить бинарную классификацию, не используя линейный слой с 2 выходными нейронами?

In [ ]:
class TorchGradientModel(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = nn.Conv2d(1, 4, kernel_size=5, padding=0, bias=True)
        self.act1 = nn.ReLU()
        self.flatten = nn.Flatten()
        self.fc1 = nn.Linear(4 * 1 * 1, 2)
        # PyTorch автоматически применяет LogSoftmax при использовании CrossEntropyloss

    def forward(self, x):
        x = self.conv1(x)
        x = self.act1(x)

        x = self.flatten(x)
        x = self.fc1(x)

        return x

Дополнительная информация может быть найдена в комментариях к коду

In [ ]:
# Данная сеть обучается за 3 эпохи, что удобно для процесса отладки.
learning_rate = 1
epochs = 3

# СТУДЕНТАМ:
# Сохраняйте историю эмпирического риска каждую эпоху в отдельном столбце loss_history 'loss_custom'
loss_history = pd.DataFrame(index=range(epochs), dtype=float)

torch_grad_model = TorchGradientModel()

# СТУДЕНТАМ:
# Используйте эти веса, чтобы инициализировать веса своей сети для точной воспроизводимости результатов
# torch_model_params[0] - тензор с весами фильтров W свёрточного слоя
# torch_model_params[1] - тензор с весами сдвигов b свёрточного слоя
# torch_model_params[2] - тензор с весами W линейного слоя
# torch_model_params[3] - тензор с весами сдвигов b линейного слоя
torch_model_params = []
temp_m = copy.deepcopy(torch_grad_model)
for param in temp_m.named_parameters():
    torch_model_params.append(param[1].clone().detach())

# СТУДЕНТАМ:
# При реализации своих слоёв не забывайте делить получившиеся градиенты по ошибкам на размер пакета (для I-го слоя - это кол-во изображений),
# чтобы эмулировать поведение CrossEntropyLoss с параметром reduction='mean'
ce = nn.CrossEntropyLoss(reduction='mean')

# Для данного эксперимента используется самый простой алгоритм обучения без моментов.
optimizer = torch.optim.SGD(torch_grad_model.parameters(), lr=learning_rate, momentum=0, dampening=0, weight_decay=0, nesterov=False)

In [ ]:
torch_grad_model.train()
t = trange(epochs)
loss_hist_pt = []
for e in t:
    predict_y = torch_grad_model(batch) # для обучения используем весь пакет
    
    # Можете выводить веса сети для прямого сравнения со своей реализацей, 
#     for param in torch_grad_model.named_parameters():
#         print(param)

    loss = ce(predict_y, labels)
    loss_history.loc[e, 'loss_pt'] = loss.item()
    
    # Градиенты нужно обнулять в каждой эпохе
    optimizer.zero_grad()
    loss.backward()
    
    # Градиенты так же можно выводить в текстовом виде для оценки хода обучения
#     print('Gradients')
#     for param in torch_grad_model.named_parameters():
#         print(param[0], param[1].grad)
    
    optimizer.step()
    
    train_acc = torch.sum(torch.argmax(predict_y, axis=1) == labels).item()

    train_acc /= batch.shape[0]
    t.set_postfix(loss=loss.item(), accuracy=train_acc)

Ниже представлены заготовки (шаблоны) классов, колторые требуется реализовать. **Conv**, **Fc** и **ReLU** должны иметь методы **forward** и **backward** для прямого и обратного прохода по сети. При прямом проходе следует кэшировать данные, которые потребуются для вычисления градиента.

**Свёртки**

При прямом проходе нужно брать фильтры поочерёдно и проводить свёртку со входным тензором. Каждая операция свёртки даёт 2-мерную матрицу на выходе. Для получения итоговой карты признаков следует сконкатенировать эти матрицы, чтобы получить тензор рамерами [Размер батча, Количество каналов, Высота, Ширина]. Реализовывать можно как с помощью вложенных циклов, так и с применением векторизации. В данном задании важно не время работы, но точность вычислений.

В первом модуле заданий уже была показана реализация обратного прохода по линейному слою для подсчёта частных производных по эмпирическому риску, которые использовались для обновления весов слоя. Расчёт частных производных в свёрточных слоях идеологически тот же. Требуется посчитать частные производные по dX предыдущему входу слоя, dW по весам фильтров и db по сдвигам. Пусть dZ – это градиент ошибки к выхожу текущего свёрточного слоя (передаётся от предыдущего слоя при обратном проходе сети), тогда

$dX += \sum_{h=0}^{n_H}\sum_{w=0}^{n_W} W_c\times dZ_{hw}$,

где $W_c$ – это фильтр, а $dZ_{hw}$ – скаляр, соответствующий градиенту эмпирического риска к выходу текущего свёрточного слоя $Z$ в $n$ строке и $w$ столбце. Так как при прямом проходе фильтр $W_c$ влияет на все значения канала с карты признаков, то мы умножаем один и тот же фильтр $W_c$ с разными $dZ$ в пределах канала $с$, суммируя результаты.
В numpy эта операция выглядела бы так

 dX[vert_start:vert_end, horiz_start:horiz_end, :] += W[:,:,:,c] * dZ[i, h, w, c]
 
В PyTorch очерёдность каналов иная: [Batch, Channel, Height, Width].

Производная одного фильтра относительно эмпирического риска  считается по формуле

$dW_c+=\sum_{h=0}^{n_H}\sum_{w=0}^{n_W}x_{slice} \times dZ_{hw}$,

где $x_{slice}$ относится к отрезку из входного тензора, который был использован в прямом проходе, чтобы получить активацию $Z_{ij}$. Таким образом мы получим градиент фильтра $W$ относительно данного отрезка. Так как в рамках свёрточного слоя для разных отрезков мы использовали тот же фильтр $W$, то мы складываем эти градиенты, чтобы получить $dW$.

В numpy подобная операция реализуется так:

dW[:,:,:,c] += x_slice * dZ[i, h, w, c]

Производная по сдвигам считается как сумма всех градиентов выхода свёрточного слоя:

$db=\sum_{h=0}^{n_H}\sum_{w=0}^{n_W}dZ_{hw}$

В numpy реализовывалась бы так:

db[:,:,:,c] += dZ[i, h, w, c]


Для реализации выполнения обратного прохода градиента идентичным nn.CrossEntropyLoss(reduction='mean') образом, значения $dW$ и $db$ следует делить на размер пакета (batch).


In [ ]:
# Наивная реализация свёртки, медленная
class Conv():
    def __init__(self, nb_filters: int, filter_size: int, nb_channels: int, stride: int = 1, padding: int = 0, sanity_check: bool = True):
        self.num_filters = nb_filters
        self.f = filter_size
        self.n_C = nb_channels
        self.s = stride
        self.p = padding
        self.sanity_check = sanity_check  # использовать не обязательно

        self.cache = None  # Для хранения данных прямого прохода сети, которые потребуются при обратном проходе
        
        self.W = torch_model_params[0]
        self.dW = torch.zeros(self.W.shape)

        
        self.b = torch_model_params[1]
        self.db =  torch.zeros(self.b.shape)

    @staticmethod
    def single_conv(img, w, b):
        # Поэлементное произведение
        s = torch.multiply(img, w)
        # сумма произведений
        g = torch.sum(s)
        # сдвиг
        g = g + b
        return g
    
    
    def forward(self, X):
        """
        Прямой проход
        - X : выход предыдущего свёрточного слоя с размерностями (m, n_C_prev, n_H_prev, n_W_prev).
        """
        raise NotImplementedError

    def backward(self, dZ):
        """
        Распространяет градиент ошибки от предыдущего слоя в текущий свёрточный слой
        - dZ: ошибка из предыдущего слоя (по направлению от выхода ко входу сети).
            
        Возвращает:
        - dX: ошибка текущего свёрточного слоя.
        - self.dW: градиент по весам.
        - self.db: градиент по сдвигам.
        """
        raise NotImplementedError
    
    
class Fc():
    def __init__(self, row, column):
        self.row = row
        self.col = column

        self.W = torch_model_params[2]
        self.dW = 0
        self.b = torch_model_params[3]
        self.db = 0
        
        self.cache = None

    def forward(self, fc):
        raise NotImplementedError

    def backward(self, dZ):
        # не забываем, что для эмуляции CrossEntropyLoss(reduction='mean') нужно делить self.dW и self.db на размер пакета
        raise NotImplementedError
    

class SGD():
    def __init__(self, lr, params):
        self.lr = lr
        self.params = params

    def update_params(self, grads):
        raise NotImplementedError


class ReLU():
    def forward(self, X):
        raise NotImplementedError
    
    def backward(self, new_deltaL):
        raise NotImplementedError

        
class Softmax():
    def __init__(self):
        pass

    def forward(self, X):
        raise NotImplementedError

        
class CrossEntropyLoss():

    def __init__(self):
        self.sm = Softmax() # Softmax не нужен в последнем слое модели сети, если вызывать в расчёте кросс-энтропии
        pass
    
    def get(self, y_pred, y):
        raise NotImplementedError

Реализовать модель, идентичную **TorchGradientModel**. 

Если есть желание переиспользовать код из первого модуля заданий по линейным слоям, то можно реализовать модель, пользуясь предыдущими абстрактными классами и импортировать их здесь, переписав код для проверки результата.

In [ ]:
class CustomModel():

    def __init__(self, input_size = 5, num_classes = 2):
        # ВПИСАТЬ МОДУЛИ СЕТИ ЗДЕСЬ
        
        self.layers = [self.conv1, self.fc1]

    def forward(self, x):
        raise NotImplementedError
        
    def backward(self, deltaL):
        # ВПИСАТЬ ОБРАТНОЕ РАСПРОСТРАНЕНИЕ ОШИБКИ
        
        grads = { 
                'dW1': dW1, 'db1': db1,
                'dW2': dW2, 'db2': db2
        }
        
        return grads


    def get_params(self):
        params = {}
        for i, layer in enumerate(self.layers):
            params['W' + str(i+1)] = layer.W
            params['b' + str(i+1)] = layer.b

        return params

    def set_params(self, params):
        for i, layer in enumerate(self.layers):
            layer.W = params['W'+ str(i+1)]
            layer.b = params['b' + str(i+1)]

In [ ]:
def train_custom_model(X, y, epochs, num_classes):
    y = one_hot_encoding(labels, num_classes) # преобразуем число в эталоне в унитарный код.
    
    model = CustomModel(input_size = X.shape[-1], num_classes=num_classes)
    cost = CrossEntropyLoss()
    
    params = model.get_params()

    optimizer = SGD(lr = learning_rate, params = model.get_params())      

    t = trange(epochs)
    
    for e in t:
        train_loss = 0
        train_acc = 0 
  
        y_pred = model.forward(X)
        loss, deltaL = cost.get(y_pred, y)
        grads = model.backward(deltaL)
        params = optimizer.update_params(grads)
        model.set_params(params)

        train_loss += loss.item()
        train_acc = torch.sum(torch.argmax(predict_y, axis=1) == labels).item() / X.shape[0]
        
        loss_history.loc[e, 'loss_custom'] = train_loss
        t.set_postfix(loss=train_loss, acc=train_acc)

In [ ]:
train_custom_model(batch, labels, epochs, num_classes=2)

In [ ]:
f = plt.figure()
plt.title('Task 1 train loss', color='black')
loss_history.plot(ax=f.gca())
plt.legend(loc='center left', bbox_to_anchor=(1.0, 0.5))
plt.show()

In [ ]:
# Эмпирический риск на всех эпохах должен совпадать с реализацией PyTorch
error = np.absolute((loss_history.iloc[:,0].values-loss_history.iloc[:,1].values)).sum()
if error < 1e-3:
    print('Задание 1 выполнено успешно')
else:
    print('Задание 1 не выполнено. Эмпирический риск не совпадает в реализациях PyTorch и собственной')
print(f'Суммарная ошибка = {error:.4f}')
loss_history_task1 = loss_history

### Задание 2

Требуется реализовать классы пакетной нормализации **BatchNorm2d** и слоя выборки усреднением **AvgPool**. Реализовать класс модели, идентичной эталонной TorchGradientModel2

Эталонная модель **TorchGradientModel2**, состоить из:
- Сверточный слой с 4 фильтрами размера $3\times3$;
- Пакетная нормализация;
- Функция активации [ReLU](https://pytorch.org/docs/stable/generated/torch.nn.ReLU.html)
- Выборки усреднением с ашгом 2 и размером окна 2
- Линейный слой с переменных количеством выходных нейронов

In [ ]:
class TorchGradientModel2(nn.Module):
    def __init__(self, num_classes=2, input_size=5):
        super().__init__()
        self.conv1 = nn.Conv2d(1, 4, kernel_size=3, padding=0, bias=True)
        self.bn1 = nn.BatchNorm2d(4)
        self.act1 = nn.ReLU()
        self.pool1 = nn.AvgPool2d(kernel_size=2, stride=2)
        self.flatten = nn.Flatten()
        self.fc1 = nn.Linear(4 * int((input_size - 2) / 2) * int((input_size - 2) / 2), num_classes)
        # PyTorch автоматически применяет LogSoftmax при использовании CrossEntropyloss

    def forward(self, x):
        x = self.conv1(x)
        x = self.bn1(x)
        x = self.act1(x)
        x = self.pool1(x)

        x = self.flatten(x)
        x = self.fc1(x)

        return x

In [ ]:
learning_rate = 1
epochs = 10

# СТУДЕНТАМ:
# Сохраняйте историю эмпирического риска каждую эпоху в отдельном столбце loss_history 'loss_custom'
loss_history = pd.DataFrame(index=range(epochs), dtype=float)

torch_grad_model = TorchGradientModel2()

# СТУДЕНТАМ:
# Используйте эти веса, чтобы инициализировать веса своей сети для точной воспроизводимости результатов
torch_model_params = []
temp_m = copy.deepcopy(torch_grad_model)
for param in temp_m.named_parameters():
    if param[0] in ('conv1.weight', 'conv1.bias', 'fc1.weight', 'fc1.bias'):
#         print(param)
        torch_model_params.append(param[1].clone().detach())

# СТУДЕНТАМ:
# При реализации своих слоёв не забывайте делить получившиеся градиенты по ошибкам на размер пакета (кол-во изображений для I-го слоя),
# чтобы эмулировать поведение CrossEntropyLoss с параметром reduction='mean'
ce = nn.CrossEntropyLoss(reduction='mean')

# Для эксперимента используется самый простой оптимизатор. При желании можете поэкспериментировать с другими, которые реализовали для 1-го задания
optimizer = torch.optim.SGD(torch_grad_model.parameters(), lr=learning_rate, momentum=0, dampening=0, weight_decay=0, nesterov=False)

torch_grad_model.train()
t = trange(epochs)
loss_hist_pt = []
for e in t:
    predict_y = torch_grad_model(batch) # для обучения используем весь пакет
    
#     Можете выводить веса сети для прямого сравнения со своей реализацей, 
#     for param in torch_grad_model.named_parameters():
#         print(param)

    loss = ce(predict_y, labels)
    loss_history.loc[e, 'loss_pt'] = loss.item()
    
    # Градиенты нужно обнулять в каждой эпохе
    optimizer.zero_grad()
    loss.backward()
    
    # Градиенты так же можно выводить в текстовом виде для оценки хода обучения
#     print('Gradients')
#     for param in torch_grad_model.named_parameters():
#         print(param[0], param[1].grad)
    
    optimizer.step()
    
    train_acc = torch.sum(torch.argmax(predict_y, axis=1) == labels).item()

    train_acc /= batch.shape[0]
    t.set_postfix(loss=loss.item(), accuracy=train_acc)

In [ ]:
class BatchNorm2d():
    def __init__(self, num_channels, gamma=1, beta=0, eps=1e-20):
        self.num_channels = num_channels
        # Применяем стандартные название полей для обновления весов, чтобы не переписывать код модели и оптимизатора
        self.W = torch.ones(num_channels)  # gamma
        self.b = torch.zeros(num_channels)  # beta
        self.eps = eps
        
        self.dW = torch.zeros(num_channels)
        self.db = torch.zeros(num_channels)

        self.cache = None
        
    def forward(self, x, debug=True):
        raise NotImplementedError
    
    def backward(self, dout):
        raise NotImplementedError


class AvgPool():
    def __init__(self, filter_size, stride):
        self.f = filter_size
        self.s = stride
        self.cache = None

    def forward(self, X):
        raise NotImplementedError

    def backward(self, dout):
        raise NotImplementedError

In [ ]:
class CustomModel2():

    def __init__(self, input_size = 5, num_classes = 2):
        # ВПИСАТЬ МОДУЛИ СЕТИ ЗДЕСЬ
        
        self.layers = [self.conv1, self.bn1, self.fc1]

    def forward(self, x):
        raise NotImplementedError
        
    def backward(self, deltaL):
        raise NotImplementedError


    def get_params(self):
        params = {}
        for i, layer in enumerate(self.layers):
            params['W' + str(i+1)] = layer.W
            params['b' + str(i+1)] = layer.b

        return params

    def set_params(self, params):
        for i, layer in enumerate(self.layers):
            layer.W = params['W'+ str(i+1)]
            layer.b = params['b' + str(i+1)]

In [ ]:
def train_custom_model2(X, y, epochs, num_classes):
    y = one_hot_encoding(labels, num_classes) # преобразуем число в эталоне в унитарный код.
    
    model = CustomModel2(input_size = X.shape[-1], num_classes=num_classes)
    cost = CrossEntropyLoss()
    
    params = model.get_params()

    optimizer = SGD(lr = learning_rate, params = model.get_params())      

    t = trange(epochs)
    
    for e in t:
        train_loss = 0
        train_acc = 0 
  
        y_pred = model.forward(X)
        loss, deltaL = cost.get(y_pred, y)
        grads = model.backward(deltaL)
        params = optimizer.update_params(grads)
        model.set_params(params)

        train_loss += loss.item()
        train_acc = torch.sum(torch.argmax(predict_y, axis=1) == labels).item() / X.shape[0]
        
        loss_history.loc[e, 'loss_custom'] = train_loss
        t.set_postfix(loss=train_loss, acc=train_acc)

In [ ]:
train_custom_model2(batch, labels, epochs, num_classes=2)

In [ ]:
f = plt.figure()
plt.title('Task 2 train loss', color='black')
loss_history.plot(ax=f.gca())
plt.legend(loc='center left', bbox_to_anchor=(1.0, 0.5))
plt.show()

In [ ]:
# Эмпирический риск на всех эпохах должен совпадать с реализацией PyTorch
error = np.absolute((loss_history.iloc[:,0].values-loss_history.iloc[:,1].values)).sum()
if error < 1e-3:
    print('Задание 2 выполнено успешно')
else:
    print('Задание 2 не выполнено. Эмпирический риск не совпадает в реализациях PyTorch и собственной')
print(f'Суммарная ошибка = {error:.4f}')
loss_history_task2 = loss_history

### Задание 3

Обучить ранее реализованную сеть **CustomModel2** на реальных данных из выборки digits.

In [ ]:
from sklearn import preprocessing
from sklearn.datasets import load_digits
from sklearn.model_selection import train_test_split
import torchvision

In [ ]:
def get_data_numbers():
    # Загружаем выборку digits
    digits = load_digits(n_class=10, return_X_y=False, as_frame=False)

    # ПРОВЕСТИ НЕОБХОДИМЫЕ ПРЕОБРАЗОВАНИЯ
    
    return train_x, train_label, val_x, val_label

In [ ]:
train_x, train_label, val_x, val_label = get_data_numbers()

In [ ]:
learning_rate = 1
epochs = 10
num_classes = 10

# СТУДЕНТАМ:
# Сохраняйте историю эмпирического риска каждую эпоху в отдельном столбце loss_history 'loss_custom'
loss_history = pd.DataFrame(index=range(epochs), dtype=float)

torch_grad_model = TorchGradientModel2(num_classes, input_size=8)

# СТУДЕНТАМ:
# Используйте эти веса, чтобы инициализировать веса своей сети для точной воспроизводимости результатов
torch_model_params = []
temp_m = copy.deepcopy(torch_grad_model)
for param in temp_m.named_parameters():
    if param[0] in ('conv1.weight', 'conv1.bias', 'fc1.weight', 'fc1.bias'):
#         print(param)
        torch_model_params.append(param[1].clone().detach())

# СТУДЕНТАМ:
# При реализации своих слоёв не забывайте делить получившиеся градиенты по ошибкам на размер пакета (кол-во изображений для I-го слоя),
# чтобы эмулировать поведение CrossEntropyLoss с параметром reduction='mean'
ce = nn.CrossEntropyLoss(reduction='mean')

# Для эксперимента используется самый простой оптимизатор. При желании можете поэкспериментировать с другими, которые реализовали для 1-го задания
optimizer = torch.optim.SGD(torch_grad_model.parameters(), lr=learning_rate, momentum=0, dampening=0, weight_decay=0, nesterov=False)

torch_grad_model.train()
t = trange(epochs)
loss_hist_pt = []
for e in t:
    predict_y = torch_grad_model(train_x[0:10]) # для обучения используем весь пакет
    
#     Можете выводить веса сети для прямого сравнения со своей реализацей, 
#     for param in torch_grad_model.named_parameters():
#         print(param)

    loss = ce(predict_y, train_label[0:10])
    loss_history.loc[e, 'loss_train_pt'] = loss.item()
    
    # Градиенты нужно обнулять в каждой эпохе
    optimizer.zero_grad()
    loss.backward()
    
    # Градиенты так же можно выводить в текстовом виде для оценки хода обучения
#     print('Gradients')
#     for param in torch_grad_model.named_parameters():
#         print(param[0], param[1].grad)
    
    optimizer.step()
    
    train_acc = torch.sum(torch.argmax(predict_y, axis=1) == train_label[0:10]).item()

    train_acc /= train_x[0:10].shape[0]
    t.set_postfix(loss=loss.item(), accuracy=train_acc)

In [ ]:
def train_model(epochs, learning_rate, loss_history, train_x, train_label, val_x, val_label):
    raise NotImplementedError

In [ ]:
# loss_history = pd.DataFrame(index=range(epochs), dtype=float)
train_model(epochs, learning_rate, loss_history, train_x[0:10], train_label[0:10], val_x[0:10], val_label[0:10])

f = plt.figure()
plt.title('Task 2 train loss', color='black')
loss_history.plot(ax=f.gca())
plt.legend(loc='center left', bbox_to_anchor=(1.0, 0.5))
plt.show()

In [ ]:
# Эмпирический риск на всех эпохах должен совпадать с реализацией PyTorch
error = np.absolute(loss_history[['loss_train_custom']].to_numpy()-loss_history[['loss_train_pt']].to_numpy()).sum()
if error < 1e-3:
    print('Задание 3 выполнено успешно')
else:
    print('Задание 3 не выполнено. Эмпирический риск не совпадает в реализациях PyTorch и собственной')
print(f'Суммарная ошибка = {error:.4f}')
loss_history_task2 = loss_history

### Необязательное задание
Создать свою архитектуру свёрточной сети и обучиться на полной выборке digits, получив высокое качество классификации. Сравнить скорость и финальное качество обучения при применении разных алгоритмов обучения (SGD с моментом, Adam), регуляризации весов.